In [ ]:
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from scipy.signal import resample

# 读取原始音频文件
data, fs = sf.read('TruE.wav')

# 如果是立体声，转换为单声道
if len(data.shape) > 1:
    print(data.shape)
    data = data.mean(axis=1)

# 生成不同采样率版本
data_4x = resample(data, 4*len(data))
data_quarter = resample(data, len(data)//4)

# 保存文件
sf.write('original.wav', data, fs)
sf.write('4x.wav', data_4x, 4*fs)
sf.write('quarter.wav', data_quarter, fs//4)

# 创建时间轴
def create_time_axis(data, fs):
    return np.arange(len(data)) / fs

# 绘制波形
plt.figure(figsize=(12, 8))

plt.subplot(3, 1, 1)
plt.plot(create_time_axis(data, fs), data/np.max(np.abs(data)))
plt.title(f'Original ({fs}Hz)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 2)
plt.plot(create_time_axis(data_4x, 4*fs), data_4x/np.max(np.abs(data_4x)))
plt.title(f'4x Sampling Rate ({4*fs}Hz)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.subplot(3, 1, 3)
plt.plot(create_time_axis(data_quarter, fs//4), data_quarter/np.max(np.abs(data_quarter)))
plt.title(f'1/4 Sampling Rate ({fs//4}Hz)')
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')

plt.tight_layout()
plt.show()

In [ ]:
def frame_signal(signal, frame_length, hop_size, window):
    frames = []
    num_frames = (len(signal) - frame_length) // hop_size + 1
    for i in range(num_frames):
        start = i * hop_size
        frame = signal[start:start+frame_length] * window
        frames.append(frame)
    return np.array(frames)

def overlap_add(frames, hop_size, window):
    frame_length = len(frames[0])
    signal_length = (len(frames)-1)*hop_size + frame_length
    reconstructed = np.zeros(signal_length)
    window_sum = np.zeros(signal_length)
    
    for i, frame in enumerate(frames):
        start = i * hop_size
        end = start + frame_length
        reconstructed[start:end] += frame
        window_sum[start:end] += window
    
    # 避免除以零
    window_sum[window_sum == 0] = 1e-12
    return reconstructed / window_sum

# 参数设置
frame_length = 1024
hop_size = 512
window = np.hamming(frame_length)

# 分帧加窗
frames = frame_signal(data, frame_length, hop_size, window)

# 恢复信号
reconstructed = overlap_add(frames, hop_size, window)

# 比较原始信号（截断到相同长度）
original = data[:len(reconstructed)]
print("恢复误差：", np.max(np.abs(original - reconstructed)))

In [ ]:
import scipy.signal as signal

# 滤波器参数
cutoff_low = 2000
cutoff_high = 4000
order = 6

# 设计滤波器
b_low, a_low = signal.butter(order, cutoff_low/(fs/2), 'low')
b_high, a_high = signal.butter(order, cutoff_high/(fs/2), 'high')

# 绘制滤波器响应
w_low, h_low = signal.freqz(b_low, a_low)
w_high, h_high = signal.freqz(b_high, a_high)

plt.figure(figsize=(12, 8))
plt.subplot(2, 2, 1)
plt.plot(w_low*fs/(2*np.pi), 20*np.log10(np.abs(h_low)))
plt.title('Lowpass Amplitude Response')
plt.ylabel('Magnitude (dB)')

plt.subplot(2, 2, 2)
plt.plot(w_low*fs/(2*np.pi), np.angle(h_low))
plt.title('Lowpass Phase Response')
plt.ylabel('Phase (rad)')

plt.subplot(2, 2, 3)
plt.plot(w_high*fs/(2*np.pi), 20*np.log10(np.abs(h_high)))
plt.title('Highpass Amplitude Response')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude (dB)')

plt.subplot(2, 2, 4)
plt.plot(w_high*fs/(2*np.pi), np.angle(h_high))
plt.title('Highpass Phase Response')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Phase (rad)')

plt.tight_layout()
plt.show()

# 应用滤波器并分析频谱
filtered_low = signal.lfilter(b_low, a_low, data)
filtered_high = signal.lfilter(b_high, a_high, data)

def plot_spectrum(signal, fs, title):
    fft = np.fft.rfft(signal)
    freq = np.fft.rfftfreq(len(signal), 1/fs)
    plt.plot(freq, 20*np.log10(np.abs(fft)))
    plt.title(title)
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Magnitude (dB)')

plt.figure(figsize=(12, 8))
plt.subplot(3, 1, 1)
plot_spectrum(data, fs, 'Original')
plt.subplot(3, 1, 2)
plot_spectrum(filtered_low, fs, 'Lowpass Filtered')
plt.subplot(3, 1, 3)
plot_spectrum(filtered_high, fs, 'Highpass Filtered')
plt.tight_layout()
plt.show()

In [ ]:
def deemphasis(signal, alpha=0.97):
    return np.append(signal[0], signal[1:] - alpha * signal[:-1])

# 应用去加重
deemphasized_audio = deemphasis(data)

# 绘制原始信号和去加重后的信号
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(np.arange(len(data)) / fs, data)
plt.title("Original Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.subplot(2, 1, 2)
plt.plot(np.arange(len(deemphasized_audio)) / fs, deemphasized_audio)
plt.title("Deemphasized Signal")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")

plt.tight_layout()
plt.show()